In [13]:
import pandas as pd
import matplotlib.pyplot as plt
from data_process import tokenizers, utils
from nanoGPT.model import  GPT, GPTConfig
from lightning_training import LightningGPT, GamesDataModule, GamesDataset
import pytorch_lightning as pl
import torch
import os
import numpy as np
import seaborn as sns
from data_process.utils import add_elo_token_to_games, join_material_tokens, remove_material_tokens

In [2]:
headers = ["index", "id", "date", "white_elo", "black_elo", "result", "ply", "ply_30s", "piece_uci"]

games_df = pd.read_csv("./data/test.csv", delimiter=";", header=None, names=headers)

games_df = games_df.sample(frac=0.1, random_state=42)

games_df

,index,id,date,white_elo,black_elo,result,ply,ply_30s,piece_uci
75721,99034768,j8dut9JL,2025.01.31,2425,2427,1-0,85,68,Pd2d4 39 39 Ng8f6 39 39 Ng1f3 39 39 Pc7c5 39 3...
80184,7380677,UMIvcBzB,2024.12.03,1919,2020,1-0,67,47,Pd2d4 39 39 Ng8f6 39 39 Nb1c3 39 39 Pc7c5 39 3...
19864,14462158,RP3reGqr,2024.12.05,2135,2144,1-0,41,-1,Ng1f3 39 39 Pc7c6 39 39 Pg2g3 39 39 Pd7d5 39 3...
76699,99355390,OC9pukWM,2025.01.31,2402,2433,1-0,53,-1,Pc2c4 39 39 Pe7e6 39 39 Nb1c3 39 39 Pd7d5 39 3...
92991,14089339,BL4die9s,2024.12.05,1200,1143,1-0,45,-1,Pe2e4 39 39 Pd7d5 39 39 Pe4d5 39 38 Qd8d5 38 3...
...,...,...,...,...,...,...,...,...,...
5002,97062,?,2021.12.07,2726,2574,1-0,9,-1,Pd2d4 39 39 Pd7d5 39 39 Nb1c3 39 39 Ng8f6 39 3...
30151,45151922,dUvdD4GK,2024.12.15,2258,2163,0-1,80,-1,Pc2c4 39 39 Pe7e6 39 39 Ng1f3 39 39 Pd7d5 39 3...
93194,14092632,7yOJ8xPr,2024.12.05,1051,1129,1-0,45,-1,Pe2e4 39 39 Pe7e5 39 39 Pd2d3 39 39 Nb8c6 39 3...
73199,98330191,Zw7vLYf0,2025.01.31,2418,2414,0-1,48,22,Pe2e4 39 39 Pd7d5 39 39 Pe4d5 39 38 Qd8d5 38 3...


In [3]:
games = join_material_tokens(games_df.piece_uci, replace_bigger_values=True)
games = add_elo_token_to_games(games, games_df.white_elo, games_df.black_elo)

In [18]:
tokenizer = tokenizers.FullMoveEloMaterialPairTokenizer()

model_config = GPTConfig(
    block_size=604,
    vocab_size=tokenizer.vocab_size,
    n_layer=8,
    n_head=8,
    n_embd=512,
    bias=False,
)

test_start_token = 21
test_token_step = 2


material_checkpoint = "./models/elo_with_material_pair/epoch=4-step=78125.ckpt"

material_model = LightningGPT.load_from_checkpoint(
    material_checkpoint,
    config=model_config,
    test_start_token=test_start_token,
    test_token_step=test_token_step,
)

base_checkpoint = "./models/elo_base/epoch=4-step=78125.ckpt"

base_model = LightningGPT.load_from_checkpoint(
    base_checkpoint,
    config=model_config,
    test_start_token=11,
    test_token_step=1,
)

number of parameters: 28.15M
number of parameters: 28.15M


In [5]:
trainer = pl.Trainer(
    accelerator="gpu",
    max_epochs=10,
    callbacks=[pl.callbacks.RichProgressBar()],
    precision="bf16-mixed",
)
torch.set_float32_matmul_precision('high')

Using bfloat16 Automatic Mixed Precision (AMP)
You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [6]:
data_module = GamesDataModule(test_games=list(games), tokenizer=tokenizer, num_workers=12)

In [7]:
data_loader = data_module.test_dataloader()

In [8]:
trainer.test(material_model, datamodule=data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.40388044714927673    │
│         test_loss         │    1.1656888723373413     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 1.1656888723373413, 'test_acc': 0.40388044714927673}]

In [17]:
base_games = remove_material_tokens(games_df.piece_uci)
base_games = add_elo_token_to_games(base_games, games_df.white_elo, games_df.black_elo)
base_games

75721    2400 2400 Pd2d4 Ng8f6 Ng1f3 Pc7c5 Pe2e3 Pb7b6 ...
80184    1900 2000 Pd2d4 Ng8f6 Nb1c3 Pc7c5 Pd4d5 Pd7d6 ...
19864    2100 2100 Ng1f3 Pc7c6 Pg2g3 Pd7d5 Pd2d3 Pe7e6 ...
76699    2400 2400 Pc2c4 Pe7e6 Nb1c3 Pd7d5 Pd2d4 Pc7c5 ...
92991    1200 1100 Pe2e4 Pd7d5 Pe4d5 Qd8d5 Nb1c3 Qd5d8 ...
                               ...                        
5002     2700 2500 Pd2d4 Pd7d5 Nb1c3 Ng8f6 Bc1f4 Pc7c5 ...
30151    2200 2100 Pc2c4 Pe7e6 Ng1f3 Pd7d5 Pg2g3 Ng8f6 ...
93194    1000 1100 Pe2e4 Pe7e5 Pd2d3 Nb8c6 Nb1c3 Ng8f6 ...
73199    2400 2400 Pe2e4 Pd7d5 Pe4d5 Qd8d5 Nb1c3 Qd5d6 ...
74479    2500 2500 Pd2d4 Ng8f6 Nb1c3 Pg7g6 Ng1f3 Pd7d5 ...
Length: 10000, dtype: object

In [19]:
base_data_module = GamesDataModule(test_games=list(base_games), tokenizer=tokenizer, num_workers=12)

In [ ]:
# model trained without material tokens is better

trainer.test(base_model, datamodule=base_data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.4135570526123047     │
│         test_loss         │    2.1594367027282715     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 2.1594367027282715, 'test_acc': 0.4135570526123047}]